In [1]:
# !pip install ollama -q
# !pip install pypdf -q

In [2]:
# !pip install langchain -q
# !pip install langchain-community -q
# !pip install chromadb -q

In [3]:
import json

In [4]:
from langchain.text_splitter import CharacterTextSplitter # dividir los textos
from langchain.document_loaders import PyPDFLoader # cargar el pdf
from langchain_community.vectorstores import Chroma # almacenar vectores de forma local
from langchain_community.embeddings import OllamaEmbeddings # crear embeddings
from langchain_core.output_parsers import StrOutputParser # texto la respuesta
from langchain_core.runnables import RunnablePassthrough # pasar la respuesta
import ollama

In [5]:
from langchain_community.llms import Ollama

In [6]:
!ollama list

NAME                  ID              SIZE      MODIFIED          
deepseek-r1:1.5b      e0979632db5a    1.1 GB    About an hour ago    
deepseek-r1:14b       c333b7232bdb    9.0 GB    5 hours ago          
llama3.1:latest       46e0c10c039e    4.9 GB    5 hours ago          
gemma3:12b            f4031aab637d    8.1 GB    5 hours ago          
gemma3:latest         a2af6cc3eb7f    3.3 GB    5 hours ago          
llama3.2:latest       a80c4f17acd5    2.0 GB    5 hours ago          
deepseek-r1:latest    6995872bfe4c    5.2 GB    5 hours ago          


In [21]:
# llm = Ollama(model="llama3.2", temperature=0.1)
llm = Ollama(model="llama3.1", temperature=0)
# llm = Ollama(model="deepseek-r1:1.5b", temperature=0.1)
# llm = Ollama(model="deepseek-r1:14b", temperature=0.1) # 8GB
# llm = Ollama(model="deepseek-r1", temperature=0.1) # 6GB

In [22]:
respuesta = llm.invoke("necesito un codigo de python para calcular la suma de dos numeros")

In [23]:
respuesta

'Aquí te dejo un ejemplo sencillo de cómo calcular la suma de dos números en Python:\n```python\n# Definimos las variables con los valores que queremos sumar\nnum1 = 5\nnum2 = 3\n\n# Calculamos la suma\nsuma = num1 + num2\n\n# Imprimimos el resultado\nprint("La suma es:", suma)\n```\nEn este ejemplo, estamos definiendo dos variables `num1` y `num2` con los valores que queremos sumar. Luego, calculamos la suma utilizando el operador `+`. Finalmente, imprimimos el resultado en la consola.\n\nSi deseas hacerlo de manera más flexible, puedes pedir al usuario que ingrese los números a sumar:\n```python\n# Pido al usuario que ingrese los números a sumar\nnum1 = float(input("Ingrese el primer número: "))\nnum2 = float(input("Ingrese el segundo número: "))\n\n# Calculo la suma\nsuma = num1 + num2\n\n# Imprimo el resultado\nprint("La suma es:", suma)\n```\nEn este caso, estamos utilizando la función `input()` para pedir al usuario que ingrese los números a sumar. La función `float()` se utiliza

In [24]:
respuesta = llm.invoke('Actúa como evaluador de objetivos. Recibirás el texto de un objetivo y debes calificar si está bien formulado según criterios SMART y de redacción. Limita tu salida exclusivamente a un JSON válido, sin texto adicional. Evalúa: 1) us o de verbos en infinitivo al inicio (por ejemplo: “Aumentar”, “Reducir”, “Implementar”); 2) si responde a qué (qué se hará), para qué (propósito o impacto) y cómo (método o acciones); 3) SMART: medible (métricas o indicadores claros con números, porcentajes o umbrales), alcanzable (factible con recursos/plazo razonable), relevante (alineado al propósito y aporta valor) y con tiempo definido (fecha, plazo o ventana temporal concreta). Si el objetivo falla en cualquiera de estos puntos, marca “aprobado” como “NO” y explica qué falta o está mal, pero no seas muy estricto de calificar son objetivos de proyectos academicos de tesis de pregrado. Devuelve 3 sugerencias de posibles objetivos escritos correctamente; si todo está correcto, deja la lista de “opciones de sugerencias” vacía. En “verbos”, lista los verbos mal escritos o que no estén en infinitivo; si no hay problemas, devuelve una lista vacía. No inventes datos que no estén en el objetivo. Formato de salida obligatorio (JSON exacto): {"aprobado": "SI" | "NO", "verbos": ["..." ], "detalle": "QUE ESTA MAL, QUE LE HACE FALTA EN CUMPLIMIENTO", "sugerencias": "QUE PUEDE MEJORAR SEMANTICAMENTE PARA SER APROBADO", "opciones de sugerencias": ["", "", ""]}. Objetivo a evaluar: """Aumentar la satisfacción del cliente en un 15% para el tercer trimestre de 2025, implementando un nuevo sistema de soporte en línea y capacitando al equipo de atención al cliente.""". Notas: No incluyas nada fuera del JSON. Si el objetivo es ambiguo, explica brevemente en “detalle” qué ambigüedades impiden la evaluación y sugiere cómo precisarlo en “sugerencias”.')

In [25]:
respuesta

'{"aprobado": "SI", "verbos": [], "detalle": "", "sugerencias": "", "opciones de sugerencias": ["Aumentar la satisfacción del cliente en un 15% para el tercer trimestre de 2025, mediante la implementación de un nuevo sistema de soporte en línea y la capacitación al equipo de atención al cliente.", "Mejorar la satisfacción del cliente en un 15% a través de la implementación de un nuevo sistema de soporte en línea y la capacitación al equipo de atención al cliente para el tercer trimestre de 2025.", "Incrementar la satisfacción del cliente en un 15% mediante la implementación de un nuevo sistema de soporte en línea y la capacitación al equipo de atención al cliente para el tercer trimestre de 2025."]}'

In [ ]:
# respuesta[respuesta.find("```json"):]

'}'

In [ ]:
# respuesta[respuesta.find("```json"):]

'}'

In [16]:
# # 1. Elimina el bloque de formato markdown y los saltos de línea innecesarios
# json_str = respuesta[respuesta.find("```json"):]
# json_str = json_str.replace("```json", "").replace("```", "").strip()

# # 2. Ahora conviértelo a un diccionario de Python
# data = json.loads(json_str)

In [17]:
data = json.loads(respuesta)

In [18]:
data

{'aprobado': 'SI',
 'verbos': [],
 'detalle': '',
 'sugerencias': '',
 'opciones de sugerencias': ['Aumentar la satisfacción del cliente en un 15% para el tercer trimestre de 2025, implementando un nuevo sistema de soporte en línea y capacitando al equipo de atención al cliente.',
  'Mejorar la satisfacción del cliente mediante la implementación de un nuevo sistema de soporte en línea y la capacitación del equipo de atención al cliente.',
  'Incrementar la satisfacción del cliente en un 15% para el tercer trimestre de 2025, a través de la implementación de un nuevo sistema de soporte en línea y la capacitación del equipo de atención al cliente.']}

### Cargar PDF

In [8]:
loader = PyPDFLoader("data/1742368983472.pdf")
paginas = loader.load()

In [9]:
len(paginas)

9

In [10]:
paginas[1].page_content

'Prompt para Aprender Inglés\nObjetivo\nMejorar mis habilidades en inglés a través de una\ncombinación de ejercicios de gramática, conversación,\ncomprensión auditiva y vocabulario. La IA actuará como mi\nprofesor, personalizando la enseñanza según mi nivel y\nestilo de aprendizaje.\nPractica tu speaking con\nChatGPT\nObjetivo\nM A R L E N E SANCHEZ\nV a n e s s a'

In [11]:
text_splitter = CharacterTextSplitter(separator="\n", chunk_size=650, chunk_overlap=80, length_function=len)

In [12]:
docs = text_splitter.split_documents(paginas)

In [13]:
len(docs)

12

In [14]:
len(paginas)

9

In [15]:
embedings = OllamaEmbeddings(model="llama3.1")
vectorstore = Chroma.from_documents(docs, embedings, collection_name="objectives_review")

C:\Users\steve\AppData\Local\Temp\ipykernel_17260\13447781.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedings = OllamaEmbeddings(model="llama3.1")


In [16]:
def ollama_llm(question, context):
    formatted_prompt = f"Pregunta: {question}\n \nContexto(responde solo sobre el contenido del contexto entregado): {context}\n"
    response = ollama.chat(model="llama3.1",
                           messages=[{"role": "user", "content": formatted_prompt}],
                           options={"temperature": 0, "max_tokens": 1000})
    return response['message']['content']


In [17]:
retriever = vectorstore.as_retriever()
def combine_documents(docs):
    return "\n\n".join([doc.page_content for doc in docs])

In [ ]:
def rag_chain(question):
    retrieved_docs = retriever.invoke(question) # extrae los vectores mas parecidos
    formatted_context = combine_documents(retrieved_docs) # combina los documentos de los que mas se parece el vector
    return ollama_llm(question, formatted_context)

In [19]:
result = rag_chain("¿Segun el texto cual es el objetivo de Prompt para Aprender Inglés?")
result

'El objetivo del Prompt para Aprender Inglés es ayudar al usuario a mejorar su nivel de inglés, ofreciendo diferentes opciones de estudio y actividades interactivas para practicar la gramática, el habla, el escucha y el vocabulario. El sistema se adapta a las necesidades y preferencias del usuario, permitiéndole elegir su estilo de aprendizaje y nivel de dificultad.'

In [20]:
result = rag_chain("¿Segun el texto como actua la IA?")
result

'Según el texto, la IA actúa como un asistente de aprendizaje de inglés que guía al usuario a través de una sesión de estudio. La IA presenta opciones y preguntas para determinar el nivel de inglés del usuario, su estilo de aprendizaje preferido y qué actividades quiere realizar durante la sesión. Luego, ofrece explicaciones y ejercicios personalizados según las opciones elegidas por el usuario.'